Author: Tarun Ruchandani (tarun.ruchandani@columbia.edu)
Date: 2016-08-02 14:26:09 -0700 (Tue, 08 Aug 2016)
Revision: 1

Spark LDA for NASA ADS v1.0 

To Do:
x Generate document term matrix for LDA
x Get LDA Results on small Corpus

8/3:

x Generate a JS Matrix per mapequation's input
- Generate topic trend reports.
o Build Mapequation with JS Matrix
o Extend to a larger corpus
o Build Docker Container
o Deploy to adsqb

- JS Matrix: figure out the right implementation - also try ifa library


8/5:

o Take care of no abstract situation
o Scale to 500 papers

8/8:

o Take care of no abstract situation
o Scale to 500 papers
o Create a graph

8/24:
o Scale to 6000 papers


In [3]:
import numpy as np
import textmining
import pyspark
import lda
import lda.datasets
import ads
import pandas as pd
from textblob import TextBlob
from numpy  import array
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import json

In [3]:
# # Extract ApJ and OpJ papers

# ads.config.token = 'TkEI7jQxScyoCtzHfhcpCWVelRqySmH5XBksQCFA'

# papers_ApJ = list(ads.SearchQuery(bibstem='ApJ',start=100,rows=10000))
# # papers_OPJ = list(ads.SearchQuery(bibstem='OPJ',start=100,rows=1000))
# # papers_PhyEs = list(ads.SearchQuery(bibstem='PhyEs',start=100,rows=1000))

# # papers = papers_ApJ+papers_OPJ+papers_PhyEs
# papers = papers_ApJ

# paper_abstracts=list()

# for paper in papers:
#     paper_abstracts.append(paper.abstract)

# # Removing papers with null abstract

# paper_abstracts = filter(None, paper_abstracts)
# paper_abstracts = list(paper_abstracts)
# print(len(paper_abstracts))


In [4]:
# Calling the API

ads.config.token = 'TkEI7jQxScyoCtzHfhcpCWVelRqySmH5XBksQCFA'

papers_ApJ = list(ads.SearchQuery(bibstem='ApJ', start=100,rows=2000, fl=['abstract','bibcode', 'title', 'body']))

papers = papers_ApJ

In [5]:
# Writing to json file
fo = open('articles.json', 'w')

for paper in papers:
    fo.write(json.dumps(paper._raw))
    fo.write("\n")
fo.close()

# and to read it
from ads.search import Article

fi = open('articles.json', 'r')
docs = []
for line in fi:
    docs.append(Article(**json.loads(line)))

print(len(docs))

2000


In [6]:
x = list(filter(lambda x: x._raw.get('abstract', None), papers))
len(x)

1477

In [7]:
paper_abstracts = list(filter(None, [x.abstract for x in docs]))
# paper_abstracts = list(docs)

print(len(paper_abstracts))

1477


/Users/tarunruchandani/anaconda3/lib/python3.5/site-packages/ads/utils.py:23: UserWarning: You are lazy loading attributes via 'abstract', and so are making multiple calls to the API. This will impact your overall rate limits.
  UserWarning,


In [8]:
from ads.search import Article
a = Article(**paper._raw)
# a = Article(author = paper._raw['author'],...)
a._raw

{'abstract': 'We investigate observational constraints on the cosmic equation of state from measurements of angular size for a large sample of milliarcsecond compact radio sources. The results are based on a flat Friedmann-Robertson-Walker (FRW) model driven by nonrelativistic matter, plus a smooth, dark energy component parameterized by its equation of state p<SUB>x</SUB>=ωρ<SUB>x</SUB> (-1〈=ω〈0). The allowed intervals for ω and Ω<SUB>m</SUB> are heavily dependent on the value of the mean projected linear size l. For l~=20-30h<SUP>-1</SUP> pc, we find Ω<SUB>m</SUB>〈=0.62, ω〈=-0.2, and Ω<SUB>m</SUB>〈=0.17, ω〈=-0.65 (68% c.l.), respectively. As a general result, this analysis shows that if one minimizes χ<SUP>2</SUP> for the parameters l, Ω<SUB>m</SUB>, and ω, the conventional flat ΛCDM model (ω=-1) with Ω<SUB>m</SUB>=0.2 and l=22.6h<SUP>-1</SUP> pc is the best fit for these angular size data.',
 'bibcode': '2002ApJ...566...15L',
 'id': '1338780',
 'title': ['Dark Energy and the Angular

In [9]:
# Build Term-Document Matrix

tdm = textmining.TermDocumentMatrix()

for paper in paper_abstracts:
    tdm.add_doc(paper)



paper_abstracts_tdm = list()

for row in tdm.rows(cutoff=0):
    paper_abstracts_tdm.append(row)


    
paper_abstracts_tdm_df = pd.DataFrame(paper_abstracts_tdm)

# Vocab of terms in abstracts
vocab = paper_abstracts_tdm_df._slice(slice(1))
vocab

d_t_freq = paper_abstracts_tdm_df._slice(slice(1,111))
# d_t_freq = paper_abstracts_tdm_df
b=d_t_freq.values
b=b.astype(int)
b.shape

(110, 10296)

In [10]:
# running LDA on TDM
# Parameters:	
# rdd – RDD of documents, which are tuples of document IDs and term (word) count vectors. 
# The term count vectors are “bags of words” with a fixed-size vocabulary 
# (where the vocabulary size is the length of the vector). Document IDs must be unique and >= 0.

# k – Number of topics to infer, i.e., the number of soft cluster centers. (default: 10)

# maxIterations – Maximum number of iterations allowed. (default: 20)

# docConcentration – Concentration parameter (commonly named “alpha”) 
# for the prior placed on documents’ distributions over topics (“theta”). (default: -1.0)

# topicConcentration – Concentration parameter (commonly named “beta” or “eta”) 
# for the prior placed on topics’ distributions over terms. (default: -1.0)

# seed – Random seed for cluster initialization. Set as None to generate seed based on system time. (default: None)

# checkpointInterval – Period (in iterations) between checkpoints. (default: 10)

# optimizer – LDAOptimizer used to perform the actual calculation. Currently “em”, “online” are supported. 
# (default: “em”)

model = lda.LDA(n_topics=20, n_iter=500, random_state=1)
model.fit(b)

topic_word = model.topic_word_  # model.components_ also works


for i, topic_dist in enumerate(topic_word):
     topic_words = np.array(vocab)


In [11]:
doc_topic = model.doc_topic_
print("type(doc_topic): {}".format(type(doc_topic)))
print("shape: {}".format(doc_topic.shape))

type(doc_topic): <class 'numpy.ndarray'>
shape: (110, 20)


In [12]:
# Document-topic Probab

doc_topic = model.doc_topic_

for n in range(5):
    sum_pr = sum(doc_topic[n,:])
    print("document: {} sum: {}".format(n, sum_pr))

for n in range(10):
    topic_most_pr = doc_topic[n].argmax()
    print("doc: {} topic: {}\n...".format(n,
                                            topic_most_pr
                                            ))


document: 0 sum: 1.0
document: 1 sum: 0.9999999999999998
document: 2 sum: 1.0
document: 3 sum: 0.9999999999999999
document: 4 sum: 0.9999999999999998
doc: 0 topic: 12
...
doc: 1 topic: 19
...
doc: 2 topic: 19
...
doc: 3 topic: 12
...
doc: 4 topic: 12
...
doc: 5 topic: 12
...
doc: 6 topic: 12
...
doc: 7 topic: 12
...
doc: 8 topic: 12
...
doc: 9 topic: 12
...


In [13]:
# Building JS Matrix - JSD Implementation 1

def multi_js(p, q):
    """Jensen-Shannon divergence (symmetric) between two multinomials,
    expressed in nats."""
    if (len(q.shape) == 2):
        axis = 1
    else:
        axis = 0
    # D_{JS}(P\|Q) = (D_{KL}(P\|Q) + D_{KL}(Q\|P)) / 2
    return 0.5 * ((q * (np.log(q.clip(1e-10,1))
                        - np.log(p.clip(1e-10,1)))).sum(axis)
                      + (p * (np.log(p.clip(1e-10,1))
                              - np.log(q.clip(1e-10,1)))).sum(axis))


JS_D1D2 = multi_js(doc_topic[0],doc_topic[1])
JS_D1D2

2.5416415599842104

In [14]:
# Alternative JS implementation

def jsd(x,y): #Jensen-shannon divergence
    import warnings
    warnings.filterwarnings("ignore", category = RuntimeWarning)
    x = np.array(x)
    y = np.array(y)
    d1 = x*np.log2(2*x/(x+y))
    d2 = y*np.log2(2*y/(x+y))
    d1[np.isnan(d1)] = 0
    d2[np.isnan(d2)] = 0
    d = 0.5*np.sum(d1+d2)    
    return d

jsd(doc_topic[0],doc_topic[1])

0.45613860245839699

In [15]:
# Constructing JS Matrix

# Creates a list containing 5 lists, each of 8 items, all set to 0
w, h = 110, 110 
Matrix = [[0 for x in range(w)] for y in range(h)] 

JSM = list()
js_df = pd.DataFrame()

for index1,item1 in enumerate(doc_topic):
    for index2,item2 in enumerate(doc_topic):
        result = jsd(doc_topic[index1],doc_topic[index2])
        Matrix[index1][index2] = result
        temp_list = list()        
        temp_list = (index1,index2,result)
        JSM.append(temp_list)

# print(JSM)

text_file3 = open("/Users/tarunruchandani/Desktop/HarvardSummer2016/Infomap/JS_output831_110.txt", "w")
text_file3.write(str(JSM))
text_file3.close()

In [16]:
# Writing to a txt file for Mapequation

text_file = open("/Users/tarunruchandani/Desktop/HarvardSummer2016/Infomap/JS_output831_110.txt", "w")

for index1,item1 in enumerate(doc_topic):
    for index2,item2 in enumerate(doc_topic):
        result = 1-(jsd(doc_topic[index1],doc_topic[index2]))
        text_file.write("%s,%s,%s\n" %(index1, index2, result))

text_file.close()



In [17]:
from random import randint

text_file_nodes = open("/Users/tarunruchandani/Desktop/HarvardSummer2016/Infomap/JS_outputNodes1831_110.txt", "w")
# group = 1

for index1 in range(0,110):
    text_file_nodes.write("%s,%s\n" %(index1, randint(0,9)))

text_file_nodes.close()



In [32]:
# Writing to a txt file for Mapequation

text_file_1554 = open("/Users/tarunruchandani/Desktop/HarvardSummer2016/Infomap/JS_output831_110_1554.txt", "w")

for index1,item1 in enumerate(doc_topic):
    for index2,item2 in enumerate(doc_topic):
        result = 1-(jsd(doc_topic[index1],doc_topic[index2]))
        result = (result - 0.8)*50
        print(result)
        
        if(result > 0):
            text_file_1554.write("%s,%s,%s\n" %(index1, index2, result))

text_file_1554.close()


10.0
-12.8069301229
-15.8768000112
-6.54389349112
-3.08139056999
-4.66089059387
-3.38506847019
-5.8527712041
-9.22771890144
-4.03737767966
-8.88583708703
-9.22322538196
-1.72890464383
-6.46105803067
-3.05531138104
-10.0971084413
-3.44762586854
0.345204541293
-4.31560055917
-4.4092693674
-7.38804610363
-3.21364975061
-7.98853146778
-4.79334100602
-6.68123877593
0.357879099171
-9.06837339993
1.64620342514
-1.2570042472
-2.0149097352
-3.75335712361
2.2719981188
-14.5325617332
-7.76792169114
-6.36570347557
-9.12353127441
-7.05342610123
-5.24471616534
-0.34547347068
-9.84167658363
-3.01011036961
-7.06743705737
-5.90291058591
3.22286958469
-7.74533812231
-6.65010680808
-2.28067843585
-5.40343352508
-7.87140848984
-8.90523749156
0.532862719382
-2.18493551753
0.431319925848
-10.933522969
-11.4799460355
-9.58835976379
-6.96170339167
0.353752791733
-8.83548703029
-5.60082351666
-2.97763164972
-10.8448087138
-3.58807577688
-8.37352530992
-8.66712326113
-4.36028724371
-8.95645215143
-3.8475691296
